### Load libraries

["Tallinn Base Map: Tallinn’s subdistrict boundaries"](http://www.tallinn.ee/est/g6497s92804)

https://ogre.adc4gis.com/


Source: [District Map of the City of Helsinki](https://hri.fi/data/en_GB/dataset/helsingin-kaupunginosat).  The maintainer of the dataset is Helsingin kaupunkiympäristön toimiala / Kaupunkimittauspalvelut. The dataset has been downloaded from [Helsinki Region Infoshare](https://hri.fi/) service on 22.11.2019 under the license [Creative Commons Attribution 4.0](https://creativecommons.org/licenses/by/4.0/).

In [1]:
import requests # library to handle requests
import pandas as pd # library for data analsysis
import numpy as np # library to handle data in a vectorized manner
import random # library for random number generation
import json # library to handle json

In [2]:
#!conda install -c conda-forge geopy --yes 

In [3]:
from geopy.geocoders import Nominatim # module to convert an address into latitude and longitude values

# libraries for displaying images
from IPython.display import Image 
from IPython.core.display import HTML 
    
# tranforming json file into a pandas dataframe library
from pandas.io.json import json_normalize
# transform geo coordinates
from pyproj import Proj, transform

In [4]:
#!conda config --set ssl_verify false 
#!conda install -c conda-forge folium=0.5.0 --yes

In [23]:
import folium # plotting library
print(folium.__version__)

0.5.0


### Define Foursquare Credentials and Version\
##### Make sure that you have created a Foursquare developer account and have your credentials handy

In [6]:
CLIENT_ID = 'WEVQCJHZEB4W2QOPCUXISUGPEB14ZDED5WSGP1TATCXL4PNR' # your Foursquare ID
CLIENT_SECRET = '2OZIDWDQUFLTEY2GUECESB11HUBKWAB1CTPVMKAHRWGCP1YK' # your Foursquare Secret
VERSION = '20191118'
LIMIT = 30
RADIUS = 1000
print('Your credentails:')
print('CLIENT_ID: ' + CLIENT_ID)
print('CLIENT_SECRET:' + CLIENT_SECRET)

Your credentails:
CLIENT_ID: WEVQCJHZEB4W2QOPCUXISUGPEB14ZDED5WSGP1TATCXL4PNR
CLIENT_SECRET:2OZIDWDQUFLTEY2GUECESB11HUBKWAB1CTPVMKAHRWGCP1YK


In [62]:
address = 'Tallinn, Etonia'

geolocator = Nominatim(user_agent="foursquare_agent")
location = geolocator.geocode(address)
latitude = location.latitude
longitude = location.longitude
print(latitude, longitude)

inProj = Proj(init='epsg:3301') # ee
inProj = Proj(init='epsg:3879') # fi
outProj = Proj(init='epsg:4326')

59.4372155 24.7453688


In [31]:
tallin_map = folium.Map(location=[latitude, longitude], zoom_start=12) # generate map centred around the Tallinn
'''
tallin_map.choropleth(geo_data='tallin01.json',
                      key_on='Feature.properties.asumi_nimi',
                      fill_color='YlOrRd', 
                      fill_opacity=0.7,
                      line_opacity=0.2,)
'''
folium.GeoJson('t02_41_asum.json', style_function=style_function).add_to(tallin_map)
tallin_map

In [36]:
#!conda install -c conda-forge pyproj --yes 

In [72]:
with open('tallin03.json', encoding='utf-8') as json_data:
    tallinn_data = json.load(json_data)

In [73]:
#spb = [x for x in features if 'Petersburg' in x['properties']['NAME_1']]
#spb[0]['properties']
features = tallinn_data['features']
features[0]
for feature in features:
    geometry = feature['geometry']
    print(geometry['type'], np.shape(geometry['coordinates']))
    Polygons = []
    if geometry['type'] == 'MultiPolygon' :
        Polygons = geometry['coordinates']
    else:
        Polygons.append(geometry['coordinates'])

    for polygon in Polygons:
        points = polygon[0]
        for point in points:
#            x1,y1 = point[0],point[1]
            point[0],point[1] = transform(inProj,outProj,point[0],point[1])

            
#        print('--', np.shape(points))


Polygon (1, 7, 2)
Polygon (1, 28, 2)
Polygon (1, 14, 2)
Polygon (1, 36, 2)
Polygon (1, 33, 2)
Polygon (1, 17, 2)
Polygon (1, 16, 2)
Polygon (1, 40, 2)
Polygon (1, 19, 2)
Polygon (1, 10, 2)
Polygon (1, 31, 2)
Polygon (1, 19, 2)
Polygon (1, 8, 2)
Polygon (1, 28, 2)
Polygon (1, 13, 2)
Polygon (1, 19, 2)
Polygon (1, 10, 2)
Polygon (1, 52, 2)
Polygon (1, 56, 2)
Polygon (1, 26, 2)
Polygon (1, 65, 2)
Polygon (1, 12, 2)
Polygon (1, 24, 2)
Polygon (1, 29, 2)
Polygon (1, 9, 2)
Polygon (1, 17, 2)
Polygon (1, 20, 2)
Polygon (1, 33, 2)
Polygon (1, 24, 2)
Polygon (1, 13, 2)
Polygon (1, 41, 2)
Polygon (1, 16, 2)
Polygon (1, 6, 2)
Polygon (1, 12, 2)
Polygon (1, 13, 2)
Polygon (1, 11, 2)
Polygon (1, 14, 2)
Polygon (1, 26, 2)
Polygon (1, 37, 2)
Polygon (1, 37, 2)
Polygon (1, 18, 2)
Polygon (1, 12, 2)
Polygon (1, 20, 2)
Polygon (1, 9, 2)
Polygon (1, 9, 2)
Polygon (1, 28, 2)
Polygon (1, 16, 2)
Polygon (1, 16, 2)
Polygon (1, 31, 2)
Polygon (1, 55, 2)
Polygon (1, 15, 2)
Polygon (1, 12, 2)
Polygon (1, 40, 2)

In [74]:
with open("tallin_geo01.json", "w", encoding='utf-8') as write_file:
    json.dump(tallinn_data, write_file, ensure_ascii=False)

In [75]:
def style_function(feature):
    return {
        'fillOpacity': 0.5,
        'weight': 0,
        'line_opacity': 0.2,
        'fillColor': '#blue' 
    }

tallin_map = folium.Map(location=[latitude, longitude], zoom_start=11) # generate map centred around the Tallinn
folium.GeoJson('tallin_geo01.json', style_function=style_function).add_to(tallin_map)
tallin_map